In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from scipy.spatial import KDTree
import numpy as np

# Additional imports
import pystac_client
import xarray as xr
import pathlib
import os
import geopandas as gpd
import matplotlib as mpl
from matplotlib import colors
import rasterio

import cartopy.crs as ccrs
import cartopy
import plotly.graph_objects as go
import plotly.express as px
from rasterio.plot import show

import matplotlib.colors as mcolors
from shapely.geometry import Point
import pandas as pd

from tqdm import tqdm

import webbrowser
from pathlib import Path
import os
import time

from itertools import compress
import pandas as pd

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

from tqdm import tqdm

from shapely.geometry import box



In [2]:
# opening STAC catalog

catalog = pystac_client.Client.open(
    #"https://storage.googleapis.com/dgds-data-public/coclico/coclico-stac/catalog.json" # cloud API
    "https://storage.googleapis.com/dgds-data-public/gca/SOTC/gca-stac_sotc/catalog.json" # P drive API 
)
#catalog

c:\Users\rowe\AppData\Local\miniconda3\envs\coclico\Lib\site-packages\pystac_client\client.py:186: NoConformsTo: Server does not advertise any conformance classes.
  warnings.warn(NoConformsTo())


In [31]:
list(catalog.get_children())

[<CollectionClient id=shore_mon_fut>,
 <CollectionClient id=sed_class>,
 <CollectionClient id=shore_mon>,
 <CollectionClient id=world_pop>,
 <CollectionClient id=world_gdp>,
 <CollectionClient id=shore_mon_hr>,
 <CollectionClient id=shore_mon_drivers>,
 <CollectionClient id=esl_gwl>,
 <CollectionClient id=sub_threat>,
 <CollectionClient id=Exp_world_pop>,
 <CollectionClient id=Exp_world_pop_parquet>,
 <CollectionClient id=Haz-ESL>,
 <CollectionClient id=Vul-Inf_Mort>,
 <CollectionClient id=Haz-GCM_90-100>,
 <CollectionClient id=Exp-NS_Slopes>,
 <CollectionClient id=Vul_MYS>,
 <CollectionClient id=Vul_RS>,
 <CollectionClient id=Haz-GCM_40-50>,
 <CollectionClient id=Haz-ERA5_04-14>,
 <CollectionClient id=Vul-GDP_Cap>,
 <CollectionClient id=Haz-GCM_04-14>,
 <CollectionClient id=Haz-Land_Sub_2010_COGs>,
 <CollectionClient id=Haz-Land_Sub_2040_COGs>,
 <CollectionClient id=Vul-Roads_Density_COGs>,
 <CollectionClient id=Exp-Delta_DEM_COGs>]

In [32]:
href = catalog.get_child("shore_mon").assets["data"].href
ds = xr.open_zarr(href)
ds

<xarray.Dataset>
Dimensions:             (stations: 1837203, time: 38)
Coordinates: (12/18)
    coastline_idint     (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    continent           (stations) |S23 dask.array<chunksize=(28707,), meta=np.ndarray>
    country             (stations) |S40 dask.array<chunksize=(28707,), meta=np.ndarray>
    country_id          (stations) |S7 dask.array<chunksize=(114826,), meta=np.ndarray>
    err_changerate      (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    err_timespan        (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    ...                  ...
    no_shorelines       (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    rmse                (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
  * time                (time) datetime64[ns] 1984-01-01 ... 2021-01-01
    timespan            (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    transect_geom       (stations) |S92 dask.array<chunksize=(14354,), meta=np.ndarray>
    transect_id         (stations) |S15 dask.array<chunksize=(57413,), meta=np.ndarray>
Dimensions without coordinates: stations
Data variables:
    changerate          (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    changerate_unc      (stations) float64 dask.array<chunksize=(57413,), meta=np.ndarray>
    outliers            (stations, time) float32 dask.array<chunksize=(114826, 3), meta=np.ndarray>
    sandy               (stations) int8 dask.array<chunksize=(459301,), meta=np.ndarray>
    sp                  (stations, time) float64 dask.array<chunksize=(57413, 3), meta=np.ndarray>
Attributes: (12/21)
    AUTHOR:              Luijendijk, A., Hagenaars, G., Ranasinghe, R., Baart...
    CITATION:            Luijendijk, A., Hagenaars, G., Ranasinghe, R. et al....
    COMMENT:             
    CRS:                 EPSG:4326
    Conventions:         CF-1.8
    DESCRIPTION:         The Shoreline Monitor is a dataset that incorporates...
    ...                  ...
    SPATIAL_EXTENT:      [-180, -90, 180, 90]
    TAGS:                ['civil engineering', 'geomorphology', 'shoreline dy...
    TEMPORAL_EXTENT:     ['1984-01-01T00:00:00Z', '2021-01-01T00:00:00Z']
    TITLE:               The Shoreline Monitor
    TITLE_ABBREVIATION:  shore_mon
    UNITS:

In [11]:
href2='https://storage.googleapis.com/dgds-data-public/coclico/shoreline_change_projections.zarr/global_shoreline_change_projections.zarr'
ds = xr.open_zarr(href2)
ds

<xarray.Dataset>
Dimensions:           (nensemble: 7, stations: 518005, nscenarios: 2, time: 2)
Coordinates:
    ensemble          (nensemble) |S3 dask.array<chunksize=(7,), meta=np.ndarray>
    lat               (stations) float64 dask.array<chunksize=(64751,), meta=np.ndarray>
    lon               (stations) float64 dask.array<chunksize=(64751,), meta=np.ndarray>
    scenarios         (nscenarios) |S5 dask.array<chunksize=(2,), meta=np.ndarray>
  * time              (time) int32 2050 2100
Dimensions without coordinates: nensemble, stations, nscenarios
Data variables:
    shoreline_change  (nscenarios, stations, time, nensemble) float64 dask.array<chunksize=(1, 64751, 1, 2), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.8
    crs:          4326
    description:  Projections of global shoreline change in view of climate c...
    source:       The procedure to produce the dataset and the findings are d...
    title:        Global shoreline change projections for the year 2050 under...

In [12]:
ds.compute()

<xarray.Dataset>
Dimensions:           (nensemble: 7, stations: 518005, nscenarios: 2, time: 2)
Coordinates:
    ensemble          (nensemble) |S3 b'1%' b'5%' b'17%' ... b'95%' b'99%'
    lat               (stations) float64 -50.38 -50.39 -50.48 ... 64.7 64.7 64.7
    lon               (stations) float64 -74.39 -74.3 -74.18 ... 39.88 39.87
    scenarios         (nscenarios) |S5 b'RCP45' b'RCP85'
  * time              (time) int32 2050 2100
Dimensions without coordinates: nensemble, stations, nscenarios
Data variables:
    shoreline_change  (nscenarios, stations, time, nensemble) float64 11.95 ....
Attributes:
    Conventions:  CF-1.8
    crs:          4326
    description:  Projections of global shoreline change in view of climate c...
    source:       The procedure to produce the dataset and the findings are d...
    title:        Global shoreline change projections for the year 2050 under...

In [13]:
ds=ds.isel(nensemble=3)


In [20]:
df=ds.to_dataframe()
df

ensemble     lat     lon scenarios  shoreline_change
stations nscenarios time                                                     
0        0          2050   b'50%' -50.378 -74.386  b'RCP45'            14.977
                    2100   b'50%' -50.378 -74.386  b'RCP45'            33.515
         1          2050   b'50%' -50.378 -74.386  b'RCP85'            14.787
                    2100   b'50%' -50.378 -74.386  b'RCP85'            32.284
1        0          2050   b'50%' -50.386 -74.303  b'RCP45'           -23.448
...                           ...     ...     ...       ...               ...
518003   1          2100   b'50%'  64.699  39.877  b'RCP85'          -190.560
518004   0          2050   b'50%'  64.697  39.874  b'RCP45'           -12.430
                    2100   b'50%'  64.697  39.874  b'RCP45'           -16.178
         1          2050   b'50%'  64.697  39.874  b'RCP85'           -58.690
                    2100   b'50%'  64.697  39.874  b'RCP85'          -169.400

[2072020 rows x 5 columns]

In [21]:
df.to_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data\01_Hazards\02_Future\Shoreline_Change\Global_Erosion_rate_FC.csv",index=True)

In [5]:
df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data\01_Hazards\02_Future\Shoreline_Change\Global_Erosion_rate_FC.csv")
df

,stations,nscenarios,time,ensemble,lat,lon,scenarios,shoreline_change
0,0,0,2050,b'50%',-50.378,-74.386,b'RCP45',14.977
1,0,0,2100,b'50%',-50.378,-74.386,b'RCP45',33.515
2,0,1,2050,b'50%',-50.378,-74.386,b'RCP85',14.787
3,0,1,2100,b'50%',-50.378,-74.386,b'RCP85',32.284
4,1,0,2050,b'50%',-50.386,-74.303,b'RCP45',-23.448
...,...,...,...,...,...,...,...,...
2072015,518003,1,2100,b'50%',64.699,39.877,b'RCP85',-190.560
2072016,518004,0,2050,b'50%',64.697,39.874,b'RCP45',-12.430
2072017,518004,0,2100,b'50%',64.697,39.874,b'RCP45',-16.178
2072018,518004,1,2050,b'50%',64.697,39.874,b'RCP85',-58.690


In [ ]:
# data_points = gpd.GeoDataFrame(
#     df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326"
# )
# data_points

In [8]:
# bounding_box=
filtered_df = df[(df['time'] == 2050) & (df['nscenarios'] == 0)]
filtered_df=filtered_df[(filtered_df["lon"]>-69.35) & (filtered_df["lon"]<-68.65) & (filtered_df["lat"]<12.45) & (filtered_df["lat"]>11.95)]
filtered_df


,stations,nscenarios,time,ensemble,lat,lon,scenarios,shoreline_change
595752,148938,0,2050,b'50%',12.200,-69.060,b'RCP45',52.3790
595756,148939,0,2050,b'50%',12.201,-69.057,b'RCP45',2.7419
595760,148940,0,2050,b'50%',12.192,-69.015,b'RCP45',86.6250
595764,148941,0,2050,b'50%',12.189,-69.013,b'RCP45',86.5500
595768,148942,0,2050,b'50%',12.184,-69.010,b'RCP45',-78.9670
...,...,...,...,...,...,...,...,...
605540,151385,0,2050,b'50%',12.204,-69.071,b'RCP45',-17.8750
605544,151386,0,2050,b'50%',12.202,-69.069,b'RCP45',14.5710
605548,151387,0,2050,b'50%',12.200,-69.065,b'RCP45',239.0400
605552,151388,0,2050,b'50%',12.198,-69.063,b'RCP45',239.0400


In [9]:

fig2 = go.Figure(go.Scattermapbox(
    lat=filtered_df["lat"],
    lon=filtered_df["lon"],
    mode='markers',
    marker=dict(
        size=15,
        color=filtered_df["shoreline_change"],  # Set color to red

        # colorscale=color_scale,  # Specify the discrete color scale
        # cmin=1,  # Set the minimum value for the color scale
        # cmax=6,  # Set the maximum value for the color scale
        colorbar=dict(
            # title=f'Curacao chainage starting counterclockwise from lon= {circle_center["geometry"][0].x:.2f} and lat= {circle_center["geometry"][0].y:.2f}'  # Set color bar title?
            title = (
                        f'Curacao RCP 4.5 2050 <br> (meters)'
                    )            
        )
    ),
    hoverinfo='text',
    hovertext=filtered_df["shoreline_change"],
    showlegend=False
))

# Update layout properties
fig2.update_layout(
    # title = f"Curacao Transect Chainage at 100m spacing starting counterclockwise from:<br> Point lon={circle_center['geometry'][0].x:.2f}° and lat={circle_center['geometry'][0].y:.2f}°",
    mapbox_style="carto-positron",
    mapbox=dict(
        zoom=9.5,
        center=dict(lat=12.18625, lon=-68.95)
    ),		

    margin=dict(l=0, r=50, b=0, t=100),
    legend=dict(
    x=1.15,
    y=1.0,
    traceorder="normal",
    orientation="v"
),
    width=800,  # Set width of the plot
    height=600  # Set height of the plot
)

# Show the plot
fig2.show()


In [ ]:
# import geopandas as gpd
# df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data\02_Exposure\Sediment_type\Global_Sed_type.csv")
# data_points = gpd.GeoDataFrame(
#     df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326"
# )
# data_points